### 비율 검정
- z-test : proportions_ztest는 두 집단의 '총 횟수'와 '성공 횟수'라는 요약본(배열)을 받아 비율의 차이를 판정한다.

In [ ]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

# 🐚 다루기 편하게 하려고 나눈거임.  
a = df.loc[df["high_price"], "Survived"] # 부자들의 생존을 따로 정리한거 0,1 구분없이
b = df.loc[~df["high_price"], "Survived"] # 그외 생존을 따로 정리한거 0,1 구분없이

count = np.array([a.sum(), b.sum()]) # 각 그룹에서 생존이 몇명인지
nobs = np.array([len(a),len(b)]) # 각 그룹 총 몇명인지

stat, p_value = proportions_ztest(count=count, nobs=nobs,alternative="two-sided") 
# alternative는 상위볼떄 Large 하위는 smaller 양쪽은 two-sided
# stat는 값 즉 z통계량 /  p-value는 확률

rate_a = a.mean()
rate_b = b.mean()
diff = rate_a - rate_b

alpha = 0.05

print("n_high:", nobs[0], "rate_high:", float(rate_a))
print("n_normal:", nobs[1], "rate_normal:", float(rate_b))
print("rate_diff(high-normal):", float(diff))
print(f"p_value: {p_value:.10f}")

if p_value < alpha:
    print("결론: (통계적으로) 두 그룹 생존률 차이가 우연으로 보기 어렵다. H0 기각")
else:
    print("결론: (통계적으로) 차이가 있다고 말하기엔 증거가 부족하다. H0 기각 못함")




n_high: 30 rate_high: 0.3
n_normal: 570 rate_normal: 0.24210526315789474
rate_diff(high-normal): 0.05789473684210525
p_value: 0.4723692779
결론: (통계적으로) 차이가 있다고 말하기엔 증거가 부족하다. H0 기각 못함


In [30]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

# A: 10번 중 8번 성공(0.8), B: 20번 중 12번 성공(0.6)
count = np.array([8, 12])
nobs = np.array([10, 20])

stat_two, p_two = proportions_ztest(count=count, nobs=nobs, alternative="two-sided")
stat_l, p_l = proportions_ztest(count=count, nobs=nobs, alternative="larger")
stat_s, p_s = proportions_ztest(count=count, nobs=nobs, alternative="smaller")

print("two-sided p:", p_two)
print("larger    p:", p_l)
print("smaller   p:", p_s)


two-sided p: 0.2733216782922979
larger    p: 0.13666083914614896
smaller   p: 0.863339160853851


# 클로드

## 1. 가설세우기
### 비즈니스 질문: "VIP 고객이 일반 고객보다 생존률이 높은가?"
- H0 (귀무가설): VIP와 일반의 생존률은 같다 (차이 = 0)
- H1 (대립가설): VIP와 일반의 생존률은 다르다 (차이 ≠ 0)

### 왜 H0가 "같다"인가?
- → 검사가 "무죄 추정"부터 시작하듯, 통계도 "효과 없음"부터 시작
- → 충분한 증거가 있어야만 "효과 있다"고 인정

## Step 2: 데이터 준비

In [ ]:
import pandas as pd
import numpy as np

# 타이타닉 데이터 로드
df = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')

# Fare 상위 10%를 VIP로 정의
threshold = df['Fare'].quantile(0.90)
df['is_vip'] = (df['Fare'] >= threshold).astype(int)

# 두 그룹 분리
vip = df[df['is_vip'] == 1]['Survived']      # VIP 그룹의 생존 여부
normal = df[df['is_vip'] == 0]['Survived']   # 일반 그룹의 생존 여부

print(f"VIP 그룹: {len(vip)}명, 생존률: {vip.mean():.1%}")
print(f"일반 그룹: {len(normal)}명, 생존률: {normal.mean():.1%}")

# 여기서 눈에 보이는 차이: 72.3% - 35.4% = 36.9%p
# 질문: 이 36.9%p 차이가 진짜야, 운이야?

VIP 그룹: 90명, 생존률: 76.7%
일반 그룹: 801명, 생존률: 34.1%


## Step 3: 검정 실행 (비율 검정)

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

# 입력 데이터 만들기
# count = [VIP 생존자 수, 일반 생존자 수]
# nobs = [VIP 총 인원, 일반 총 인원]

count = np.array([vip.sum(), normal.sum()])  # [64, 284] 같은 형태
nobs = np.array([len(vip), len(normal)])     # [89, 802] 같은 형태

print(f"VIP: {count[0]}명 생존 / {nobs[0]}명 = {count[0]/nobs[0]:.1%}")
print(f"일반: {count[1]}명 생존 / {nobs[1]}명 = {count[1]/nobs[1]:.1%}")

# Z-test 실행
z_stat, p_value = proportions_ztest(count, nobs)

print(f"\n검정통계량(z): {z_stat:.3f}")
print(f"p-value: {p_value:.6f}")

# **예상 출력:**
# ```
# VIP: 64명 생존 / 89명 = 71.9%
# 일반: 284명 생존 / 802명 = 35.4%

# 검정통계량(z): 6.847
# p-value: 0.000000  (실제로는 7.5e-12 같은 극소값)

VIP: 69명 생존 / 90명 = 76.7%
일반: 273명 생존 / 801명 = 34.1%

검정통계량(z): 7.876
p-value: 0.000000


## Step 4: 판결 내리기


In [ ]:
alpha = 0.05  # 유의수준 (기준선)

if p_value < alpha:
    print(f"✅ p-value({p_value:.4f}) < {alpha}")
    print("→ 귀무가설(H0) 기각!")
    print("→ VIP와 일반의 생존률 차이는 통계적으로 유의미하다.")
else:
    print(f"❌ p-value({p_value:.4f}) >= {alpha}")
    print("→ 귀무가설(H0) 기각 실패")
    print("→ 차이가 우연일 가능성을 배제할 수 없다.")

✅ p-value(0.0000) < 0.05
→ 귀무가설(H0) 기각!
→ VIP와 일반의 생존률 차이는 통계적으로 유의미하다.


## Step 5: 결론 작성법 (실무 템플릿)

In [35]:
# 효과 크기 (얼마나 차이나는지)
effect = vip.mean() - normal.mean()

print("=" * 50)
print("[분석 결론]")
print(f"• 효과 크기: VIP가 일반보다 {effect:.1%}p 높은 생존률")
print(f"• 통계적 유의성: p-value = {p_value:.2e} < 0.05")
print(f"• 샘플 수: VIP {len(vip)}명, 일반 {len(normal)}명")
print(f"• 판정: 차이는 우연이 아니라 유의미함")
print("=" * 50)

[분석 결론]
• 효과 크기: VIP가 일반보다 42.6%p 높은 생존률
• 통계적 유의성: p-value = 3.37e-15 < 0.05
• 샘플 수: VIP 90명, 일반 801명
• 판정: 차이는 우연이 아니라 유의미함
